# Travel Advisor Code

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from pprint import pprint
from config import api_key
import requests

In [2]:

#Travel Advisor
lat_url = "https://travel-advisor.p.rapidapi.com/restaurants/list-by-latlng"

lat = 43.65107
long = -79.347015

lat_querystring = {"latitude":lat,"longitude":long,"limit":"30","currency":"CAD","distance":"2","lunit":"km","lang":"en_US"}

headers = {
"X-RapidAPI-Key": api_key,
"X-RapidAPI-Host": "travel-advisor.p.rapidapi.com"
}

response = requests.get(lat_url, headers=headers, params=lat_querystring)


location_id = []
res_name = []
res_add = []
review_dates = []
count = 0

for i in range(0,30):
    location_id.append(response.json()["data"][i]["location_id"])

loc_url = "https://travel-advisor.p.rapidapi.com/restaurants/get-details"

for loc_id in location_id:
    loc_querystring = {"location_id":loc_id,"currency":"CAD","lang":"en_US"}

    resturants = requests.get(loc_url, headers=headers, params=loc_querystring)

    #count+=1
    #print(count)
    #pprint(resturants.json())
    
    if "address" in resturants.json():
        res_name.append(resturants.json()["name"])
        res_add.append(resturants.json()["address"])
        
        if int(resturants.json()["num_reviews"]) > 0:
            review_dates.append(resturants.json()["reviews"])
        else:
            review_dates.append("")
    
    
 
# print(len(res_name))
# print(len(res_add))
# print(len(review_dates))

data = {
    "Name" : res_name,
    "Address" : res_add,
    "Reviews" : review_dates
    
    
}


df = pd.DataFrame(data)

df


,Name,Address,Reviews
0,Keating Channel Pub and Grill,"2 Villiers St, Toronto, Ontario M5A 1B1 Canada","[{'title': 'A great spot.', 'rating': '5', 'pu..."
1,Sukhothai,"490 Front St E, Toronto, Ontario M5A 0J1 Canada","[{'title': 'Amazing food', 'rating': '4', 'pub..."
2,The Aviary,"484A Front St E, Toronto, Ontario M5A 0J1 Canada","[{'title': 'Such a nice and cozy visit', 'rati..."
3,Power Kitchen,"28 Logan Ave, Toronto, Ontario M4M 2M8 Canada",
4,District 28,"28 Logan Ave, Toronto, Ontario M4M 2M8 Canada","[{'title': 'Amazing space with fast internet, ..."
5,Cherry Street Bar-B-Que,"275 Cherry St, Toronto, Ontario M5A 3L3 Canada","[{'title': 'THEY MADE MY DAY', 'rating': '5', ..."
6,Vivo Pizza + Pasta,"460 Front St E, Toronto, Ontario M5A 0E6 Canada",[{'title': 'Terrific place for pasts and pizza...
7,Saulter Street Brewery,"31 Saulter Street, Toronto, Ontario M4M 2H8 Ca...",
8,Southern Crown Caribbean Smokehouse,"100 Broadview Avenue, Toronto, Ontario M4M 3H3...",
9,Animal Liberation Kitchen,"100 Broadview Ave #202, Toronto, Ontario M4M 3...","[{'title': 'Vegan for the animals', 'rating': ..."
